<a href="https://colab.research.google.com/github/dmw1998/Case_Study_Log/blob/main/Full_model_One_PCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install casadi
!pip install chaospy

In [2]:
import casadi as ca
import chaospy as cp
import numpy as np
import matplotlib.pyplot as plt

In [12]:
dist = cp.Normal(1.0, 1.0)
poly_basis = cp.expansion.stieltjes(5, dist, retall=False)
nodes, weights = cp.generate_quadrature(5, dist, rule='gaussian')

print(poly_basis[1]([1.0, 2.0, 3.9]))
print(nodes)
print(weights)

[0.  1.  2.9]
[[-2.32425743 -0.88917588  0.38329341  1.61670659  2.88917588  4.32425743]]
[0.00255578 0.08861575 0.40882847 0.40882847 0.08861575 0.00255578]


In [125]:
def solve_ocp_pce(pce_order=5, k_mean=1.0, k_std=0.25):
    # Time and discretization
    tf = 40        # final time [sec]
    N = 80         # number of control intervals
    dt = tf / N    # time step

    # Aircraft physical constants
    m = 4662                # mass [lb sec^2 / ft]
    g = 32.172              # gravity [ft/sec^2]
    delta = 0.03491         # thrust inclination angle [rad]

    # Thrust model coefficients: T = A0 + A1*V + A2*V^2
    A0 = 0.4456e5           # [lb]
    A1 = -0.2398e2          # [lb sec / ft]
    A2 = 0.1442e-1          # [lb sec^2 / ft^2]

    # Aerodynamic model
    rho = 0.2203e-2         # air density [lb sec^2 / ft^4]
    S = 0.1560e4            # reference surface area [ft^2]

    # Wind model 3 beta (smoothing) parameters
    beta0 = 0.4             # initial beta value (approximate)
    beta_dot0 = 0.2         # initial beta rate
    sigma = 3               # time to reach beta = 1 [sec]

    # C_D(alpha) = B0 + B1 * alpha + B2 * alpha**2, D = 0.5 * C_D(α) * ρ * S * V²
    B0 = 0.1552
    B1 = 0.12369            # [1/rad]
    B2 = 2.4203             # [1/rad^2]

    # Lift coefficient: C_L = C0 + C1 * alpha (+ C2 * alpha**2)
    C0 = 0.7125             # baseline lift coefficient
    C1 = 6.0877             # AOA lift slope [1/rad]

    # Lift/drag model optional extensions (if needed)
    C2 = -9.0277            # [rad^-2] — e.g., for moment or drag extension

    # Angle of attack & control constraints
    umax = 0.05236          # max control input (rate of change of alpha) [rad/sec]
    alphamax = 0.3          # max angle of attack [rad]
    alpha_star = 0.20944    # changing pt of AoA

    # Wind model x parameters (piecewise smooth wind)
    a = 6e-8                 # x transition midpoint [ft]
    b = -4e-11               # second transition point [ft]

    # Wind model h parameters (polynomial form)
    c = -np.log(25/30.6)*1e-12      # transition smoothing width [ft]
    d = -8.02881e-8         # polynomial coeff [sec^-1 ft^-2]
    e = 6.28083e-11         # polynomial coeff [sec^-1 ft^-3]

    # Cost function / target altitude
    hR = 1000               # reference altitude [ft]
    h_star = 1000           # used in some wind models

    # Auxiliary
    eps = 1e-6              # to avoid division by zero in V

    # Scaling factors (used if normalizing states)
    xscale = 10000          # [ft]
    hscale = 1000           # [ft]
    Vscale = 240            # [ft/sec]
    gammascale = 0.1        # [rad]
    alphascale = 0.3        # [rad]
    uscale = 0.05           # [rad/sec]

    # PCE parameters
    dist = cp.Normal(k_mean, k_std)
    poly_basis = cp.expansion.stieltjes(pce_order, dist, retall=False)
    nodes, weights = cp.generate_quadrature(pce_order, dist, rule='gaussian')
    nodes = nodes.flatten()
    M = len(nodes)    # M = pce_order + 1

    Psi_mat = np.zeros((M, M))
    for i in range(M):
        Psi_mat[:, i] = poly_basis[i](nodes)

    Psi_dm = ca.DM(Psi_mat)
    weights_dm = ca.DM(weights).T

    # Opti instance and scaled variables
    opti = ca.Opti()
    x_s = opti.variable(M, N+1)
    h_s = opti.variable(M, N+1)
    V_s = opti.variable(M, N+1)
    gamma_s = opti.variable(M, N+1)
    alpha_s = opti.variable(M, N+1)
    X_pce = opti.variable(5*(pce_order+1), N+1)
    u_s = opti.variable(N)

    # Unscaled variables for dynamics
    x = x_s * xscale
    h = h_s * hscale
    V = V_s * Vscale
    gamma = gamma_s * gammascale
    alpha = alpha_s * alphascale
    u = u_s * uscale

    def Smooth(x_, x0, x1):
        t = (x_ - x0) / (x1 - x0 + eps)
        return ca.if_else(x_ < x0, 0,
               ca.if_else(x_ > x1, 1, 6*t**5 - 15*t**4 + 10*t**3))

    def A_piecewise(x_):
        A1 = -50 + a * x_**3 + b * x_**4
        A2 = 0.025 * (x_ - 2300)
        A3 = 50 - a * (4600 - x_)**3 - b * (4600 - x_)**4
        A4 = 50
        s1 = Smooth(x_, 480, 520)
        s2 = Smooth(x_, 4080, 4120)
        s3 = Smooth(x_, 4580, 4620)
        B12 = (1 - s1)*A1 + s1*A2
        B23 = (1 - s2)*A2 + s2*A3
        B34 = (1 - s3)*A3 + s3*A4
        return ca.if_else(x_ <= 500, B12,
               ca.if_else(x_ <= 4100, B23,
               ca.if_else(x_ <= 4600, B34, A4)))

    def B_piecewise(x_):
        B1 = d * x_**3 + e * x_**4
        B2 = -51 * ca.exp(ca.fmin(-c * (x_ - 2300)**4, 30))
        B3 = d * (4600 - x_)**3 + e * (4600 - x_)**4
        B4 = 0
        s1 = Smooth(x_, 480, 520)
        s2 = Smooth(x_, 4080, 4120)
        s3 = Smooth(x_, 4580, 4620)
        B12 = (1 - s1)*B1 + s1*B2
        B23 = (1 - s2)*B2 + s2*B3
        B34 = (1 - s3)*B3 + s3*B4
        return ca.if_else(x_ <= 500, B12,
               ca.if_else(x_ <= 4100, B23,
               ca.if_else(x_ <= 4600, B34, B4)))

    def wind_x(x_, k_):
        return k_ * A_piecewise(x_)

    def wind_h(x_, h_, k_):
        h_safe = ca.fmax(h_, 10.0)
        return k_ * h_safe / h_star * B_piecewise(x_)

    def C_L(alpha_):
        return ca.if_else(alpha_ > alpha_star, C0 + C1 * alpha_,
                          C0 + C1 * alpha_ + C2 * (alpha_ - alpha_star)**2)
    def beta(t_):
        return ca.if_else(t_ < sigma, beta0 + beta_dot0 * t_, 1.0)

    # Symbolic derivatives
    x_sym = ca.MX.sym("x")
    h_sym = ca.MX.sym("h")
    k_sym = ca.MX.sym("k")
    Wx_expr = wind_x(x_sym, k_sym)
    Wh_expr = wind_h(x_sym, h_sym, k_sym)
    dWx_dx_fun = ca.Function("dWx_dx", [x_sym, k_sym], [ca.gradient(Wx_expr, x_sym)])
    dWh_dx_fun = ca.Function("dWh_dx", [x_sym, h_sym, k_sym], [ca.gradient(Wh_expr, x_sym)])
    dWh_dh_fun = ca.Function("dWh_dh", [x_sym, h_sym, k_sym], [ca.gradient(Wh_expr, h_sym)])

    def aircraft_ode(X, u_, t_, k_val):
        x_, h_, V_, gamma_, alpha_ = ca.vertsplit(X)

        T = beta(t_) * (A0 + A1 * V_ + A2 * V_**2)
        D = 0.5 * (B0 + B1 * alpha_ + B2 * alpha_**2) * rho * S * V_**2
        L = 0.5 * rho * S * C_L(alpha_) * V_**2

        Wx = wind_x(x_, k_val)
        Wh = wind_h(x_, h_, k_val)
        V_safe = ca.fmax(V_, 1e-3)

        x_dot = V_ * ca.cos(gamma_) + Wx
        h_dot = V_ * ca.sin(gamma_) + Wh

        dWx_dx_val = dWx_dx_fun(x_, k_val)[0]
        dWh_dx_val = dWh_dx_fun(x_, h_, k_val)[0]
        dWh_dh_val = dWh_dh_fun(x_, h_, k_val)[0]

        Wx_dot = dWx_dx_val * x_dot
        Wh_dot = dWh_dx_val * x_dot + dWh_dh_val * h_dot

        V_dot = T / m * ca.cos(alpha_ + delta) - D / m - g * ca.sin(gamma_) - (Wx_dot * ca.cos(gamma_) + Wh_dot * ca.sin(gamma_))
        gamma_dot = T / (m * V_safe) * ca.sin(alpha_ + delta) + L / (m * V_safe) - g / V_safe * ca.cos(gamma_) + (1 / V_safe) * (Wx_dot * ca.sin(gamma_) - Wh_dot * ca.cos(gamma_))
        alpha_dot = u_

        return ca.vertcat(x_dot, h_dot, V_dot, gamma_dot, alpha_dot)

    def rk4_step(f, xk, uk, tk, dt, k_val):
        k1 = f(xk, uk, tk, k_val)
        k2 = f(xk + dt/2 * k1, uk, tk + dt/2, k_val)
        k3 = f(xk + dt/2 * k2, uk, tk + dt/2, k_val)
        k4 = f(xk + dt * k3, uk, tk + dt, k_val)
        return xk + dt/6 * (k1 + 2*k2 + 2*k3 + k4)

    # # Integration using RK4
    # X_sym = ca.MX.sym("X", 5)
    # u_sym = ca.MX.sym("u")

    J = 0

    # Initial conditions
    opti.subject_to(x_s[:, 0] == 0)
    opti.subject_to(h_s[:, 0] == 600 / hscale)
    opti.subject_to(V_s[:, 0] == 239.7 / Vscale)
    opti.subject_to(gamma_s[:, 0] == -0.03925 / gammascale)
    opti.subject_to(alpha_s[:, 0] == 0.1283 / alphascale)
    opti.subject_to(ca.vec(V_s) >= 1e-2 / Vscale)
    opti.subject_to(gamma_s[:, -1] == -0.05236 / gammascale)

    for i in range(N):
        h_physical = h_s[:, i] * hscale
        h_mean = ca.mtimes(weights_dm, h_physical)
        deviation = hR - h_mean
        scaled_deviation = deviation / hscale
        J += dt * scaled_deviation**6

        for j, k_val in enumerate(nodes):
            tk = i * dt     # New line
            Xk = ca.vertcat(x[j, i], h[j, i], V[j, i], gamma[j, i], alpha[j, i])
            Uk = u[i]
            Xk_end = rk4_step(aircraft_ode, Xk, Uk, tk, dt, k_val)
            X_next = ca.vertcat(x[j, i+1], h[j, i+1], V[j, i+1], gamma[j, i+1], alpha[j, i+1])
            opti.subject_to(X_next == Xk_end)

        opti.subject_to(opti.bounded(-1, u_s[i], 1))
        alpha_i = alpha_s[:, i] * alphascale
        alpha_pce_coeffs = ca.mtimes(weights_dm * alpha_i.T, Psi_dm)
        alpha_high_coeffs = alpha_pce_coeffs[1:]
        alpha_mean = ca.mtimes(weights_dm, alpha_i)
        # std_term = ca.sumsqr(alpha_high_coeffs)
        std_term = ca.sqrt(ca.sumsqr(alpha_high_coeffs))

        opti.subject_to(alpha_mean + 3 * std_term <= alphamax)
        opti.subject_to(alpha_mean - 3 * std_term >= -alphamax)

    # # Cost function
    # J = dt * ca.sumsqr((h_s - (hR / hscale))**3)
    opti.minimize(J)

    # Initial guess
    for j in range(M):
        opti.set_initial(x_s[j, :], np.linspace(0, 1, N+1))
        opti.set_initial(h_s[j, :], 0.6)  # 600 ft / 1000
        opti.set_initial(V_s[j, :], 239.7 / Vscale)
        opti.set_initial(gamma_s[j, :], -0.01 / gammascale)
        opti.set_initial(alpha_s[j, :], 0.02 / alphascale)
    opti.set_initial(u_s, 0)

    # Solver
    opts = {
        "expand": True,
        "ipopt": {
            "max_iter": 3000,
            "tol": 1e-6,
            "print_level": 5,
            "linear_solver": "mumps",
            "hessian_approximation": "limited-memory",
            # "bound_push": 1e-8,
            # "bound_frac": 1e-8
        }
    }
    opti.solver("ipopt", opts)

    try:
        sol = opti.solve()
    except RuntimeError as e:
        opti.debug.show_infeasibilities()
        print(e)
        return {
            "x": opti.debug.value(x),
            "h": opti.debug.value(h),
            "V": opti.debug.value(V),
            "gamma": opti.debug.value(gamma),
            "alpha": opti.debug.value(alpha),
            "u": opti.debug.value(u),
            "J": opti.debug.value(J),
            "status": "failed"
        }

    return {
        "x": sol.value(x),
        "h": sol.value(h),
        "V": sol.value(V),
        "gamma": sol.value(gamma),
        "alpha": sol.value(alpha),
        "u": sol.value(u),
        "J": sol.value(J),
        "status": "success"
    }

In [126]:
res = solve_ocp_pce(pce_order=2, k_mean=1.0, k_std=0.25)

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     7458
Number of nonzeros in inequality constraint Jacobian.:      803
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     1295
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1218
Total number of inequality constraints...............:      483
        inequality constraints with only lower bounds:      323
   inequality constraints with lower and upper bounds:       80
        inequality constraints with only upper bounds:       80

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.6384000e-01 4.09e+01 1.00e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

CasADi - 2025-06-03 20:49:12 WARNING("KeyboardInterruptException") [.../casadi/interfaces/ipopt/ipopt_nlp.cpp:132]


0), in order of declaration:
------- i = 0/1701 ------ 
0 <= 5.20325e-09 <= 0 (viol 5.20325e-09)
Opti constraint of shape 3x1, called from /content/<ipython-input-125-ee68c6e62b6b>:198 in solve_ocp_pce
At nonzero 0, part 0.
------- i = 1/1701 ------ 
0 <= 1.25981e-07 <= 0 (viol 1.25981e-07)
Opti constraint of shape 3x1, called from /content/<ipython-input-125-ee68c6e62b6b>:198 in solve_ocp_pce
At nonzero 1, part 0.
------- i = 2/1701 ------ 
0 <= -7.44135e-07 <= 0 (viol 7.44135e-07)
Opti constraint of shape 3x1, called from /content/<ipython-input-125-ee68c6e62b6b>:198 in solve_ocp_pce
At nonzero 2, part 0.
------- i = 3/1701 ------ 
0.6 <= 0.6 <= 0.6 (viol 2.30781e-08)
Opti constraint of shape 3x1, called from /content/<ipython-input-125-ee68c6e62b6b>:199 in solve_ocp_pce
At nonzero 0, part 0.
------- i = 4/1701 ------ 
0.6 <= 0.6 <= 0.6 (viol 1.45668e-07)
Opti constraint of shape 3x1, called from /content/<ipython-input-125-ee68c6e62b6b>:199 in solve_ocp_pce
At nonzero 1, part 0.
---

In [131]:
import numpy as np
import casadi as ca
import chaospy as cp

def solve_ocp_pce_fast(pce_order=5, k_mean=1.0, k_std=0.25, N=80, tf=40, verbose=True):
    # 常量定义略（和你原版一致）...
    m = 4662; g = 32.172; delta = 0.03491
    A0 = 0.4456e5; A1 = -0.2398e2; A2 = 0.1442e-1
    rho = 0.2203e-2; S = 0.1560e4
    B0 = 0.1552; B1 = 0.12369; B2 = 2.4203
    C0 = 0.7125; C1 = 6.0877; C2 = -9.0277
    xscale, hscale, Vscale, gammascale, alphascale, uscale = 10000, 1000, 240, 0.1, 0.3, 0.05
    hR = 1000; alphamax = 0.3; alpha_star = 0.20944
    dt = tf / N

    # 风模型参数
    beta0 = 0.4; beta_dot0 = 0.2; sigma = 3
    a = 6e-8; b = -4e-11
    c = -np.log(25 / 30.6) * 1e-12
    d = -8.02881e-8; e = 6.28083e-11
    h_star = 1000

    eps = 1e-6

    # --- PCE节点和权重 ---
    dist = cp.Normal(k_mean, k_std)
    nodes, weights = cp.generate_quadrature(pce_order, dist, rule='gaussian')
    nodes = nodes.flatten(); weights = weights / np.sum(weights)
    M = len(nodes)

    # --- CasADi Opti变量 (批量) ---
    opti = ca.Opti()
    x_s = opti.variable(M, N+1)
    h_s = opti.variable(M, N+1)
    V_s = opti.variable(M, N+1)
    gamma_s = opti.variable(M, N+1)
    alpha_s = opti.variable(M, N+1)
    u_s = opti.variable(N)

    # --- 初始条件 ---
    opti.subject_to(x_s[:, 0] == 0)
    opti.subject_to(h_s[:, 0] == 600 / hscale)
    opti.subject_to(V_s[:, 0] == 239.7 / Vscale)
    opti.subject_to(gamma_s[:, 0] == -0.03925 / gammascale)
    opti.subject_to(alpha_s[:, 0] == 0.1283 / alphascale)
    opti.subject_to(ca.vec(V_s) >= 1e-2 / Vscale)
    opti.subject_to(gamma_s[:, -1] == -0.05236 / gammascale)

    # ------ 你的风模型定义 -------
    def Smooth(x_, x0, x1):
        t = (x_ - x0) / (x1 - x0 + eps)
        return ca.if_else(x_ < x0, 0,
               ca.if_else(x_ > x1, 1, 6*t**5 - 15*t**4 + 10*t**3))
    def A_piecewise(x_):
        A1 = -50 + a * x_**3 + b * x_**4
        A2 = 0.025 * (x_ - 2300)
        A3 = 50 - a * (4600 - x_)**3 - b * (4600 - x_)**4
        A4 = 50
        s1 = Smooth(x_, 480, 520)
        s2 = Smooth(x_, 4080, 4120)
        s3 = Smooth(x_, 4580, 4620)
        B12 = (1 - s1)*A1 + s1*A2
        B23 = (1 - s2)*A2 + s2*A3
        B34 = (1 - s3)*A3 + s3*A4
        return ca.if_else(x_ <= 500, B12,
               ca.if_else(x_ <= 4100, B23,
               ca.if_else(x_ <= 4600, B34, A4)))
    def B_piecewise(x_):
        B1 = d * x_**3 + e * x_**4
        B2 = -51 * ca.exp(ca.fmin(-c * (x_ - 2300)**4, 30))
        B3 = d * (4600 - x_)**3 + e * (4600 - x_)**4
        B4 = 0
        s1 = Smooth(x_, 480, 520)
        s2 = Smooth(x_, 4080, 4120)
        s3 = Smooth(x_, 4580, 4620)
        B12 = (1 - s1)*B1 + s1*B2
        B23 = (1 - s2)*B2 + s2*B3
        B34 = (1 - s3)*B3 + s3*B4
        return ca.if_else(x_ <= 500, B12,
               ca.if_else(x_ <= 4100, B23,
               ca.if_else(x_ <= 4600, B34, B4)))
    def wind_x(x_, k_):
        return k_ * A_piecewise(x_)
    def wind_h(x_, h_, k_):
        h_safe = ca.fmax(h_, 10.0)
        return k_ * h_safe / h_star * B_piecewise(x_)

    def C_L(alpha_):
        return ca.if_else(alpha_ > alpha_star, C0 + C1 * alpha_,
                          C0 + C1 * alpha_ + C2 * (alpha_ - alpha_star)**2)
    def beta(t_): return ca.if_else(t_ < sigma, beta0 + beta_dot0 * t_, 1.0)

    # --- 动力学 ---
    def aircraft_ode(x_, h_, V_, gamma_, alpha_, u_, t_, k_):
        T = beta(t_) * (A0 + A1 * V_ + A2 * V_**2)
        D = 0.5 * (B0 + B1 * alpha_ + B2 * alpha_**2) * rho * S * V_**2
        L = 0.5 * rho * S * C_L(alpha_) * V_**2
        Wx = wind_x(x_, k_)
        Wh = wind_h(x_, h_, k_)
        V_safe = ca.fmax(V_, 1e-3)
        x_dot = V_ * ca.cos(gamma_) + Wx
        h_dot = V_ * ca.sin(gamma_) + Wh
        V_dot = T / m * ca.cos(alpha_ + delta) - D / m - g * ca.sin(gamma_)
        gamma_dot = T / (m * V_safe) * ca.sin(alpha_ + delta) + L / (m * V_safe) - g / V_safe * ca.cos(gamma_)
        alpha_dot = u_
        return ca.vertcat(x_dot, h_dot, V_dot, gamma_dot, alpha_dot)

    def rk4_vec(Xk, Uk, tk, dt, k_):
        f = lambda X, u, t, k: aircraft_ode(X[0], X[1], X[2], X[3], X[4], u, t, k)
        k1 = f(Xk, Uk, tk, k_)
        k2 = f(Xk + dt/2 * k1, Uk, tk + dt/2, k_)
        k3 = f(Xk + dt/2 * k2, Uk, tk + dt/2, k_)
        k4 = f(Xk + dt * k3, Uk, tk + dt, k_)
        return Xk + dt/6 * (k1 + 2*k2 + 2*k3 + k4)

    for i in range(N):
        for j in range(M):
            Xk = ca.vertcat(x_s[j, i]*xscale, h_s[j, i]*hscale, V_s[j, i]*Vscale, gamma_s[j, i]*gammascale, alpha_s[j, i]*alphascale)
            Uk = u_s[i]*uscale
            tk = i*dt
            k_val = nodes[j]
            Xk_end = rk4_vec(Xk, Uk, tk, dt, k_val)
            X_next = ca.vertcat(x_s[j, i+1]*xscale, h_s[j, i+1]*hscale, V_s[j, i+1]*Vscale, gamma_s[j, i+1]*gammascale, alpha_s[j, i+1]*alphascale)
            opti.subject_to(X_next == Xk_end)
            opti.subject_to(opti.bounded(-1, u_s[i], 1))
            opti.subject_to(opti.bounded(-1, alpha_s[j, i], 1))
            opti.subject_to(V_s[j, i] >= 1e-2 / Vscale)

    # --- 只在终端加统计约束 ---
    weights_dm = ca.DM(weights).T
    alpha_final = alpha_s[:, -1] * alphascale
    mean_alpha = ca.mtimes(weights_dm, alpha_final)  # (1,M) x (M,1)
    std_alpha = ca.sqrt(ca.mtimes(weights_dm, (alpha_final - mean_alpha)**2))
    opti.subject_to(mean_alpha + 3 * std_alpha <= alphamax)
    opti.subject_to(mean_alpha - 3 * std_alpha >= -alphamax)

    h_final = h_s[:, -1] * hscale
    mean_h = ca.mtimes(weights_dm, h_final)
    obj = (mean_h - hR)**6
    opti.minimize(obj)

    # --- 初值warm start ---
    for j in range(M):
        opti.set_initial(x_s[j, :], np.linspace(0, 1, N+1))
        opti.set_initial(h_s[j, :], 0.6)
        opti.set_initial(V_s[j, :], 239.7 / Vscale)
        opti.set_initial(gamma_s[j, :], -0.01 / gammascale)
        opti.set_initial(alpha_s[j, :], 0.02 / alphascale)
    opti.set_initial(u_s, 0)

    opts = {"expand": True, "ipopt": {"max_iter": 1000, "tol": 1e-6, "print_level": 5}}
    opti.solver("ipopt", opts)
    sol = opti.solve()

    return {
        "x": sol.value(x_s) * xscale,
        "h": sol.value(h_s) * hscale,
        "V": sol.value(V_s) * Vscale,
        "gamma": sol.value(gamma_s) * gammascale,
        "alpha": sol.value(alpha_s) * alphascale,
        "u": sol.value(u_s) * uscale,
    }

In [132]:
res1 = solve_ocp_pce_fast(pce_order=2, k_mean=1.0, k_std=0.25)

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     6258
Number of nonzeros in inequality constraint Jacobian.:      969
Number of nonzeros in Lagrangian Hessian.............:     4892

Total number of variables............................:     1295
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1218
Total number of inequality constraints...............:      965
        inequality constraints with only lower bounds:      484
   inequality constraints with lower and upper bounds:      480
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.0960000e+15 4.09e+01 4.10e+08  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:217:
.../casadi/core/optistack_internal.cpp:1334: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Error_In_Step_Computation'